In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)

import shared_utils
import pandas as pd
import geopandas as gpd

import gcsfs
from calitp_data_analysis import get_fs
from calitp_data_analysis import geography_utils, utils
fs = get_fs()
import re
import google.auth
import os
import gcsfs
credentials, project = google.auth.default()
fs = gcsfs.GCSFileSystem()

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
GCS_FILE_PATH  = 'gs://calitp-analytics-data/data-analyses/ahsc_grant/'

In [ ]:
job_density_prv = pd.read_csv(f"{GCS_FILE_PATH}Job Density Raw File Private.csv")

In [ ]:
job_density_tot = pd.read_csv(f"{GCS_FILE_PATH}Job Density Raw File Total.csv")

In [ ]:
job_density_prv = job_density_prv.rename(columns={'C000': 'jobs_prv'})
job_density_tot = job_density_tot.rename(columns={'C000': 'jobs_tot'})

In [ ]:
job_density_prv.info()

In [ ]:
job_density_tot.info()

In [ ]:
merged_df = pd.merge(
    job_density_tot,
    job_density_prv[['w_geocode', 'jobs_prv']],
    on='w_geocode',
    how='left'
)

In [ ]:
#Sort by GEOID
merged_df = merged_df.sort_values(by='w_geocode')

#Create jobs_fed = jobs_total - jobs_prv
merged_df['jobs_fed'] = merged_df['jobs_tot'] - merged_df['jobs_prv']


In [ ]:
merged_df['GEOID'] = merged_df['w_geocode'].astype(str).str.zfill(15).str[:11]

In [ ]:
grouped_df = merged_df.groupby('GEOID', as_index=False).agg({
    'jobs_tot': 'sum',
    'jobs_prv': 'sum',
    'jobs_fed': 'sum',
    'year': 'first'
})

In [ ]:
def export_gdf(gdf, filename: str):
    
    gdf.to_parquet(f"{filename}.parquet")
    
    fs.put(
        f"{filename}.parquet",
        f"{GCS_FILE_PATH}/{filename}.parquet",
        token = credentials.token
    )
    
    os.remove(f"{filename}.parquet")
    print(f"saved {GCS_FILE_PATH}/{filename}.parquet")
    
    return

In [ ]:
export_gdf(grouped_df, "job_density_2022")